In [1]:
import numpy as np
import pandas as pd
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /home/nanwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nanwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('/home/nanwang/github/python-algrothm/laioffer/notes/watch_reviews.tsv',sep='\t',error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [3]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [4]:
df.dropna(subset=['review_body'],inplace=True)

In [5]:
df.reset_index(inplace=True, drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [7]:
# use the first 1000 data as our training data
data = df.loc[:999, 'review_body'].tolist()
# data = df.loc[:, 'review_body'].tolist()

In [8]:
data

f it was going to work well.<br />I'm  saying this because it was  ridiculous low price for the look that this product can give. surprisingly enough this watch works fine. except for the 24 hr  and montly indicators. they are stationary.  i though the metal part wast going to rust and change color, so far it hasn't .<br />good product for that price.",
 'A little small',
 'Nice',
 'I love it.',
 'excellent',
 'Excellent watch and very fast delivery.  Very pleased',
 "This is a very nice watch. I like to use it when I am working in the garden and in the kitchen. I don't have to worry about having to take it off for dirty work.",
 "This is my second G shock and it's really a cool one. It's a little more expensive than the main line of Gshocks but it more than makes up for it with its appearance. As others have said here, the colors are not pure orange and blue, but a tad washed out. The Japanese special edition has more vibrant color. I knew this before hand and chose this version as I t

In [9]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [10]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [11]:
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'wear',
 'dainti']

In [12]:
data[0]

'Absolutely love this watch! Get compliments almost every time I wear it. Dainty.'

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 1000 reviews and 239 terms.


In [14]:
tfidf_matrix

<1000x239 sparse matrix of type '<class 'numpy.float64'>'
	with 6891 stored elements in Compressed Sparse Row format>

In [15]:
tfidf_matrix.toarray()

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [16]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [17]:
# print out words
tf_selected_words

['abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi',
 'high',
 'hold',


In [18]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [19]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [20]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,0
1,I love this watch it keeps time wonderfully.,0
2,Scratches,3
3,"It works well on me. However, I found cheaper ...",3
4,Beautiful watch face. The band looks nice all...,3
5,"i love this watch for my purpose, about the pe...",0
6,"for my wife and she loved it, looks great and ...",1
7,I was about to buy this thinking it was a Swis...,3
8,Watch is perfect. Rugged with the metal &#34;B...,1
9,Great quality and build.<br />The motors are r...,3


In [21]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
3,646
0,138
1,80
4,74
2,62


In [22]:
km.cluster_centers_

# 239数的list -> cluster 0的中心点的tf-idf值
#-> assumption: 中心点的值可以代表这个cluster
#-> tf-idf值越大，对应的词越能代表这个document
#-> 选出了tf-idf最大的6个值对应的词来代表这个cluster

array([[0.        , 0.04120682, 0.        , ..., 0.00983041, 0.01569585,
        0.00457865],
       [0.00403114, 0.        , 0.        , ..., 0.        , 0.        ,
        0.0148226 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00784247,
        0.        ],
       [0.0058403 , 0.00280065, 0.00395318, ..., 0.00708839, 0.018194  ,
        0.01513846],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00886222,
        0.        ]])

In [23]:
km.cluster_centers_.shape

(5, 239)

In [24]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

t...<br />I would give this watch 5 stars if not for the QC issues with chapter ring alignment., Its a bad ass watch., VERYYYYY GOOOOOD, Looks good and fits great.  It is thin, so I am not sure how durable it is, but it might be that I am used to the canvas style rather than the nylon.  I like how it looks on my Seiko, and will wear it as soon as I replace the battery (in my watch, the strap does not require one)., Product not as described - bezel is black not gunmetal like the face.  I live overseas and due to postage costs this is too cheap a watch to bother returning, but disappointed I didn't get what I paid for., I have to admit that iI wasn't quite sure I was actually going to get it first of all and that if I did if it was going to work well.<br />I'm  saying this because it was  ridiculous low price for the look that this product can give. surprisingly enough this watch works fine. except for the 24 hr  and montly indicators. they are stationary.  i though the metal part wast g

In [25]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [26]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.0604675  0.06014371 0.0601367  0.75948254 0.05976955]
 [0.65898564 0.08533355 0.08653628 0.08445737 0.08468717]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10000027 0.10011124 0.10007594 0.10000022 0.59981233]
 [0.19833652 0.06856935 0.24843735 0.417546   0.06711078]
 [0.06723789 0.71777973 0.07222505 0.07551428 0.06724305]]


In [27]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [28]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.06016931 0.54106347 0.05986289 0.27869087 0.06021346]
 [0.08497771 0.65416636 0.08839356 0.08736519 0.08509718]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10009964 0.10000028 0.59982001 0.10007918 0.1000009 ]
 [0.06902197 0.505778   0.28988028 0.06819445 0.0671253 ]
 [0.07001656 0.06736165 0.06723462 0.07178947 0.7235977 ]]


In [29]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 239)
[[1.01908761 0.2007403  0.20283815 ... 0.20225849 0.75443271 0.20016072]
 [0.20031885 7.36405777 0.20013627 ... 2.61701132 2.16830328 2.47927666]
 [0.20004466 0.20145129 0.20064762 ... 0.20124301 0.20102925 0.76303085]
 [3.47401193 0.20387449 2.7473079  ... 3.71083057 6.49478225 5.49505224]
 [0.20186403 0.52563898 0.20282514 ... 0.2043539  6.4428403  3.65958498]]


In [30]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.06,0.54,0.06,0.28,0.06,1
Doc1,0.08,0.65,0.09,0.09,0.09,1
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.17,0.06,0.13,0.06,0.59,4
Doc4,0.04,0.04,0.04,0.39,0.49,4
Doc5,0.07,0.69,0.08,0.07,0.08,1
Doc6,0.41,0.39,0.06,0.06,0.06,0
Doc7,0.06,0.06,0.74,0.07,0.07,2
Doc8,0.04,0.20,0.05,0.66,0.05,3
Doc9,0.06,0.06,0.26,0.57,0.06,3


In [31]:
df_document_topic['topic'].value_counts().to_frame()

,topic
3,253
4,224
0,181
2,176
1,166


In [32]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[1.01908761 0.2007403  0.20283815 ... 0.20225849 0.75443271 0.20016072]
 [0.20031885 7.36405777 0.20013627 ... 2.61701132 2.16830328 2.47927666]
 [0.20004466 0.20145129 0.20064762 ... 0.20124301 0.20102925 0.76303085]
 [3.47401193 0.20387449 2.7473079  ... 3.71083057 6.49478225 5.49505224]
 [0.20186403 0.52563898 0.20282514 ... 0.2043539  6.4428403  3.65958498]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,...,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,1.019088,0.200740,0.202838,0.204510,1.297699,0.203161,0.202793,0.201840,0.201165,0.201399,...,0.201806,0.200198,11.836352,0.200100,0.200153,1.649044,0.200093,0.202258,0.754433,0.200161
Topic1,0.200319,7.364058,0.200136,0.201997,0.200454,0.200224,0.200154,0.200066,9.711550,0.200980,...,0.200134,0.200085,0.201398,0.200568,0.200030,0.201874,0.807418,2.617011,2.168303,2.479277
Topic2,0.200045,0.201451,0.200648,0.201501,0.201188,0.200930,0.952575,0.200701,0.202693,3.380079,...,0.200824,0.204182,0.200455,0.200311,0.200128,31.972090,0.200324,0.201243,0.201029,0.763031
Topic3,3.474012,0.203874,2.747308,3.403024,0.916835,3.665266,4.206335,2.673701,0.449846,2.812676,...,0.867777,0.632660,0.201754,3.429364,3.769889,6.107691,0.909811,3.710831,6.494782,5.495052
Topic4,0.201864,0.525639,0.202825,1.845364,3.676995,1.670830,0.201512,1.563695,0.203580,0.207405,...,4.247822,2.480049,0.202439,0.203365,0.200308,1.130968,2.085447,0.204354,6.442840,3.659585


In [33]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,nice,price,excel,expect,beauti,wife,realli,deliveri,pleas,look,valu,clock,everyth,product,fast
Topic 1,love,great,look,amaz,big,pictur,beauti,feel,absolut,husband,super,fit,better,exact,comfort
Topic 2,good,work,product,perfect,qualiti,great,batteri,time,ship,use,day,deal,bad,stop,arriv
Topic 3,like,look,awesom,time,pretti,want,broke,wear,week,color,band,watch,love,face,compliment
Topic 4,band,strap,gift,look,water,came,make,hand,buy,larg,size,littl,like,replac,time
